# 02 - Feature Engineering

This notebook focuses on extracting and engineering features from the transit data.


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import sys

# Add src to path
sys.path.append(str(Path('../src').resolve()))
from features import extract_time_features, extract_route_features, extract_traffic_features

# Load processed data
data_dir = Path('../data/processed')
# Add your feature extraction code here
